In [12]:
import os
import ipywidgets
import adaptive_mis as mis
import adaptive_mis.common.jupyter_utils as ju
from adaptive_mis.common import load_config
from adaptive_mis.common import printc
if 'datasets' not in os.listdir('.'):
    os.chdir('../')

CONFIG_ROOT="./configs"
DATASETS_ROOT=f"{CONFIG_ROOT}/datasets"
MODELS_ROOT=f"{CONFIG_ROOT}/models"
ADAPTIVE_LAYERS_ROOT=f"{MODELS_ROOT}/adaptive"
EVAL_ROOT=f'{CONFIG_ROOT}/evaluation'

all_datasets=sorted(os.listdir(DATASETS_ROOT))
all_datasets.insert(0,'select dataset')
dataset_dd=ipywidgets.Dropdown(options=all_datasets,description="dataset")
display(dataset_dd)
ju.wait_for_user(dataset_dd)
dataset_cfg=f'{DATASETS_ROOT}/{dataset_dd.value}'
print(f'selected dataset={dataset_cfg}')

Dropdown(description='dataset', options=('select dataset', 'isic.yaml'), value='select dataset')

...............................continuing...
selected dataset=./configs/datasets/isic.yaml


In [10]:
all_adaptive_layers=sorted(os.listdir(ADAPTIVE_LAYERS_ROOT))
all_adaptive_layers.insert(0,"No Adaptive Layer")
adaptive_dd=ipywidgets.Dropdown(options=all_adaptive_layers,description="Adaptive Layer (optional)")
display(adaptive_dd)
all_models=sorted([f for f in os.listdir(MODELS_ROOT) if f.endswith(".yaml")])
all_models.insert(0,'select model')
model_dd=ipywidgets.Dropdown(options=all_models,description="Main Model")
display(model_dd)
ju.wait_for_user(model_dd)
adaptive_layer_cfg=f'{ADAPTIVE_LAYERS_ROOT}/{adaptive_dd.value}' if "No Adaptive Layer" not in  adaptive_dd.value else None
main_model_cfg=f'{MODELS_ROOT}/{model_dd.value}'

print("Selected Adaptive Layer", adaptive_layer_cfg)
print("Selected Main Model", main_model_cfg)

Dropdown(description='Adaptive Layer (optional)', options=('No Adaptive Layer', 'acda.yaml', 'adaptive.yaml'),…

Dropdown(description='Main Model', options=('select model', 'acda_uctransnet.yaml', 'adapt_uctransnet.yaml', '…

...............................................................continuing...
Selected Adaptive Layer ./configs/models/adaptive/acda.yaml
Selected Main Model ./configs/models/multiresunet.yaml


In [13]:
all_evaluations=sorted([f for f in os.listdir(EVAL_ROOT) if f.endswith(".yaml")])
all_evaluations.insert(0,'select evaluation')
eval_dd=ipywidgets.Dropdown(options=all_evaluations,description="Evaluation")
display(eval_dd)
ju.wait_for_user(eval_dd)
eval_cfg=f'{EVAL_ROOT}/{eval_dd.value}'
print("Selected Evaluation", eval_cfg)

Dropdown(description='Evaluation', options=('select evaluation', 'demo.yaml', 'kfold.yaml', 'split.yaml'), val…

.................................continuing...
Selected Evaluation ./configs/evaluation/demo.yaml


In [18]:
from datetime import datetime
import os
def basename(path):
    return os.path.basename(path).split('.')[0]


if adaptive_layer_cfg:
    model_cfg=f"""
model: 
    class: "adaptive_mis.models.adaptive.AdaptiveModel"
    title: "AdaptCustom"
    load_weights: false
    params:
        adaptive_layer: !include {adaptive_layer_cfg}
        main_model: !include {main_model_cfg}
"""
    model_title=f"Adapt-{basename(adaptive_layer_cfg)}-{basename(main_model_cfg)}"
else:
    model_cfg=f"""
model: !include {main_model_cfg}
"""
    model_title=basename(main_model_cfg)
    
date = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
log = f'{basename(dataset_cfg)}/{model_title}-{date}'
config=f"""
dataset: !include {dataset_cfg}
evaluation: !include {eval_cfg}
run:
    comet: True
    save_dir: ./results/{log}
{model_cfg}
"""

    
print(config)


cfg=load_config(f"!include {CONFIG_ROOT}/common.yaml")
cfg.update(load_config(config))
import json
print(json.dumps(cfg, indent=2))


dataset: !include ./configs/datasets/isic.yaml
evaluation: !include ./configs/evaluation/demo.yaml
run:
    comet: True
    save_dir: ./results/isic/Adapt:acda-multiresunet-2023-09-20T13-40-10

model: 
    class: "adaptive_mis.models.adaptive.AdaptiveModel"
    title: "AdaptCustom"
    load_weights: false
    params:
        adaptive_layer: !include ./configs/models/adaptive/acda.yaml
        main_model: !include ./configs/models/multiresunet.yaml


{
  "data_loader": {
    "train": {
      "batch_size": 16,
      "shuffle": true,
      "num_workers": 8,
      "pin_memory": true
    },
    "validation": {
      "batch_size": 16,
      "shuffle": false,
      "num_workers": 8,
      "pin_memory": true
    },
    "test": {
      "batch_size": 16,
      "shuffle": false,
      "num_workers": 4,
      "pin_memory": false
    }
  },
  "training": {
    "optimizer": {
      "name": "Adam",
      "params": {
        "lr": 0.0001
      }
    },
    "criterion": {
      "name": "DiceLoss",
   

In [20]:
from adaptive_mis import pipeline

pipeline.execute(cfg)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Torch device: cpu
{
  "data_loader": {
    "train": {
      "batch_size": 16,
      "shuffle": true,
      "num_workers": 8,
      "pin_memory": true
    },
    "validation": {
      "batch_size": 16,
      "shuffle": false,
      "num_workers": 8,
      "pin_memory": true
    },
    "test": {
      "batch_size": 16,
      "shuffle": false,
      "num_workers": 4,
      "pin_memory": false
    }
  },
  "training": {
    "optimizer": {
      "name": "Adam",
      "params": {
        "lr": 0.0001
      }
    },
    "criterion": {
      "name": "DiceLoss",
      "params": {}
    },
    "scheduler": {
      "factor": 0.5,
      "patience": 10
    },
    "epochs": 3
  },
  "dataset": {
    "title": "ISIC2018",
    "class": "adaptive_mis.dataset.ISIC2018Dataset",
    "params": {
      "datadir": "datasets/ISIC2018",
      "image_size": 224
    }
  },
  "evaluation": {
    "class": "adaptive_mis.evaluation.Split",
    "title": "Demo",
    "params": {
      "seed": 1,
      "slice": 0.03,
    

COMET INFO: Experiment is live on comet.com https://www.comet.com/modaresimr/isic2018/aaff2d97aef2407fb77897c63c80e58c



NotADirectoryError: [WinError 267] The directory name is invalid: './results/isic/Adapt:acda-multiresunet-2023-09-20T13-40-10'